In [268]:
import subprocess
import os
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
import random
import time
import pickle

In [269]:
# #--------------------------intra frame prediction-----------------------------------#
# import cv2
# import numpy as np
# from google.colab.patches import cv2_imshow

# def intra_frame_prediction(current_frame):
#     # Convert the frame to grayscale
#     gray_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

#     # Get the frame dimensions
#     height, width = gray_frame.shape

#     # Create an empty array for the predicted frame
#     predicted_frame = np.zeros_like(gray_frame)

#     # Iterate over blocks (e.g., 4x4 blocks) and perform spatial prediction
#     block_size = 4
#     for y in range(0, height, block_size):
#         for x in range(0, width, block_size):
#             # Get the current block
#             current_block = gray_frame[y:y+block_size, x:x+block_size]

#             # Predict the block based on neighboring pixels (simple averaging)
#             prediction = np.mean(current_block)

#             # Fill the predicted block in the predicted frame
#             predicted_frame[y:y+block_size, x:x+block_size] = prediction

#     return predicted_frame

# # Example usage
# current_frame = cv2.imread("/content/frames/frame_0000.jpg")

# predicted_frame = intra_frame_prediction(current_frame)

# # Display the original and predicted frames for visual comparison in Colab
# cv2_imshow(current_frame)
# cv2_imshow(predicted_frame)



In [270]:
# #---------------------Inter frame prediction-----------------------------------------#
# import cv2
# import numpy as np
# from google.colab.patches import cv2_imshow

# def inter_frame_prediction(prev_frame, current_frame, next_frame):
#     # Convert frames to grayscale
#     prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
#     current_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
#     next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

#     # Compute motion vectors between frames (previous to current and current to next)
#     flow_prev_current = cv2.calcOpticalFlowFarneback(prev_gray, current_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
#     flow_current_next = cv2.calcOpticalFlowFarneback(current_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

#     # Apply motion vectors to predict the current frame
#     predicted_current_frame = cv2.remap(current_frame, flow_prev_current, None, cv2.INTER_LINEAR)
#     predicted_next_frame = cv2.remap(current_frame, flow_current_next, None, cv2.INTER_LINEAR)

#     # Interpolate between the two predictions to get the final predicted frame
#     alpha = 0.5  # Interpolation factor
#     final_predicted_frame = cv2.addWeighted(predicted_current_frame, alpha, predicted_next_frame, 1 - alpha, 0)

#     return final_predicted_frame

# # Example usage
# prev_frame = cv2.imread("/content/frames/frame_0000.jpg")
# current_frame = cv2.imread("/content/frames/frame_0002.jpg")
# next_frame = cv2.imread("/content/frames/frame_0005.jpg")

# predicted_frame = inter_frame_prediction(prev_frame, current_frame, next_frame)

# # Display the original and predicted frames for visual comparison
# cv2_imshow( current_frame)
# cv2_imshow( predicted_frame)



In [271]:
# #---------------------------motion vector prediction---------------------------------#
# import cv2
# import numpy as np

# def motion_vector_prediction(prev_frame, current_frame):
#     # Convert frames to grayscale
#     prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
#     current_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

#     # Calculate dense optical flow using Farneback method
#     flow = cv2.calcOpticalFlowFarneback(prev_gray, current_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

#     # Get the motion vectors for each pixel
#     motion_vectors = flow.reshape(-1, 2)

#     return motion_vectors

# # Example usage
# prev_frame = cv2.imread("/content/frames/frame_0000.jpg")
# current_frame = cv2.imread("/content/frames/frame_0200.jpg")

# motion_vectors = motion_vector_prediction(prev_frame, current_frame)

# # Display the original and motion vector prediction for visual comparison

# cv2_imshow( current_frame)

# # Draw motion vectors on the current frame for visualization
# for vector in motion_vectors:
#     start_point = tuple(vector.astype(int))
#     end_point = tuple((vector + [10, 10]).astype(int))  # Adjust the length of the vector for visualization
#     cv2.arrowedLine(current_frame, start_point, end_point, (0, 255, 0), 1)

# cv2_imshow( current_frame)



In [272]:
#------------------spliting as frames--------------------------#

def split_video_frames(input_video, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Define the FFmpeg command to extract frames at 1 frame per second
    ffmpeg_command = [
        'ffmpeg',
        '-i', input_video,
        '-vf', 'fps=10',
        os.path.join(output_folder, 'frame_%4d.jpg')
    ]

    # Execute the FFmpeg command
    subprocess.run(ffmpeg_command)

# Example usage
input_video = "/content/mov_2.mov"
output_folder = "/content/frames_jpg_8"

split_video_frames(input_video, output_folder)




In [273]:
# Code to Perform Block Matching
debug = True

def YCrCb2BGR(image):
    """
    Converts numpy image into from YCrCb to BGR color space
    """
    return cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

def BGR2YCrCb(image):
    """
    Converts numpy image into from BGR to YCrCb color space
    """
    return cv2.cvtColor(image, cv2.COLOR_YCrCb2BGR)

def segmentImage(anchor, blockSize=16):
    """
    Determines how many macroblocks an image is composed of
    :param anchor: I-Frame
    :param blockSize: Size of macroblocks in pixels
    :return: number of rows and columns of macroblocks within
    """
    h, w = anchor.shape
    hSegments = int(h / blockSize)
    wSegments = int(w / blockSize)
    totBlocks = int(hSegments * wSegments)

    #if debug:
    #    print(f"Height: {h}, Width: {w}")
    #    print(f"Segments: Height: {hSegments}, Width: {wSegments}")
    #    print(f"Total Blocks: {totBlocks}")

    return hSegments, wSegments

def getCenter(x, y, blockSize):
    """
    Determines center of a block with x, y as top left corner coordinates and blockSize as blockSize
    :return: x, y coordinates of center of a block
    """
    return (int(x + blockSize/2), int(y + blockSize/2))

def getAnchorSearchArea(x, y, anchor, blockSize, searchArea):
    """
    Returns image of anchor search area
    :param x, y: top left coordinate of macroblock in Current Frame
    :param anchor: I-Frame
    :param blockSize: size of block in pixels
    :param searchArea: size of search area in pixels
    :return: Image of anchor search area
    """
    h, w = anchor.shape
    cx, cy = getCenter(x, y, blockSize)

    sx = max(0, cx-int(blockSize/2)-searchArea) # ensure search area is in bounds
    sy = max(0, cy-int(blockSize/2)-searchArea) # and get top left corner of search area

    # slice anchor frame within bounds to produce anchor search area
    anchorSearch = anchor[sy:min(sy+searchArea*2+blockSize, h), sx:min(sx+searchArea*2+blockSize, w)]

    return anchorSearch

def getBlockZone(p, aSearch, tBlock, blockSize):
    """
    Retrieves the block searched in the anchor search area to be compared with the macroblock tBlock in the current frame
    :param p: x,y coordinates of macroblock center from current frame
    :param aSearch: anchor search area image
    :param tBlock: macroblock from current frame
    :param blockSize: size of macroblock in pixels
    :return: macroblock from anchor
    """
    px, py = p # coordinates of macroblock center
    px, py = px-int(blockSize/2), py-int(blockSize/2) # get top left corner of macroblock
    px, py = max(0,px), max(0,py) # ensure macroblock is within bounds

    aBlock = aSearch[py:py+blockSize, px:px+blockSize] # retrive macroblock from anchor search area


    try:
        assert aBlock.shape == tBlock.shape # must be same shape

    except Exception as e:
        print(e)
        print(f"ERROR - ABLOCK SHAPE: {aBlock.shape} != TBLOCK SHAPE: {tBlock.shape}")

    return aBlock

def getMAD(tBlock, aBlock):
    """
    Returns Mean Absolute Difference between current frame macroblock (tBlock) and anchor frame macroblock (aBlock)
    """
    return np.sum(np.abs(np.subtract(tBlock, aBlock)))/(tBlock.shape[0]*tBlock.shape[1])

def getBestMatch(tBlock, aSearch, blockSize): #3 Step Search
    """
    Implemented 3 Step Search. Read about it here: https://en.wikipedia.org/wiki/Block-matching_algorithm#Three_Step_Search
    :param tBlock: macroblock from current frame
    :param aSearch: anchor search area
    :param blockSize: size of macroblock in pixels
    :return: macroblock from anchor search area with least MAD
    """
    step = 4
    ah, aw = aSearch.shape
    acy, acx = int(ah/2), int(aw/2) # get center of anchor search area

    minMAD = float("+inf")
    minP = None

    while step >= 1:
        p1 = (acx, acy)
        p2 = (acx+step, acy)
        p3 = (acx, acy+step)
        p4 = (acx+step, acy+step)
        p5 = (acx-step, acy)
        p6 = (acx, acy-step)
        p7 = (acx-step, acy-step)
        p8 = (acx+step, acy-step)
        p9 = (acx-step, acy+step)
        pointList = [p1,p2,p3,p4,p5,p6,p7,p8,p9] # retrieve 9 search points

        for p in range(len(pointList)):
            aBlock = getBlockZone(pointList[p], aSearch, tBlock, blockSize) # get anchor macroblock
            MAD = getMAD(tBlock, aBlock) # determine MAD
            if MAD < minMAD: # store point with minimum mAD
                minMAD = MAD
                minP = pointList[p]

        step = int(step/2)

    px, py = minP # center of anchor block with minimum MAD
    px, py = px - int(blockSize / 2), py - int(blockSize / 2) # get top left corner of minP
    px, py = max(0, px), max(0, py) # ensure minP is within bounds
    matchBlock = aSearch[py:py + blockSize, px:px + blockSize] # retrieve best macroblock from anchor search area

    return matchBlock



def blockSearchBody(anchor, target, blockSize, searchArea=7):
    """
    Facilitates the creation of a predicted frame based on the anchor and target frame
    :param anchor: I-Frame
    :param target: Current Frame to create a P-Frame from
    :param blockSize: size of macroBlock in pixels
    :param searchArea: size of searchArea extended from blockSize
    :return: predicted frame
    """
    h, w = anchor.shape
    hSegments, wSegments = segmentImage(anchor, blockSize)


    predicted = np.ones((h, w))*255
    bcount = 0
    for y in range(0, int(hSegments*blockSize), blockSize):
        for x in range(0, int(wSegments*blockSize), blockSize):
            bcount+=1
            targetBlock = target[y:y+blockSize, x:x+blockSize] #get current macroblock

            anchorSearchArea = getAnchorSearchArea(x, y, anchor, blockSize, searchArea) #get anchor search area

            #print("AnchorSearchArea: ", anchorSearchArea.shape)

            anchorBlock = getBestMatch(targetBlock, anchorSearchArea, blockSize) #get best anchor macroblock
            predicted[y:y+blockSize, x:x+blockSize] = anchorBlock #add anchor block to predicted frame

            #cv2.imwrite("OUTPUT/predictedtestFrame.png", predicted)
            #print(f"ITERATION {bcount}")

    #cv2.imwrite("OUTPUT/predictedtestFrame.png", predicted)

    #time.sleep(10)

    assert bcount == int(hSegments*wSegments) #check all macroblocks are accounted for

    return predicted

def getResidual(target, predicted):
    """Create residual frame from target frame - predicted frame"""
    return np.subtract(target, predicted)

def getReconstructTarget(residual, predicted):
    """Reconstruct target frame from residual frame plus predicted frame"""
    return np.add(residual, predicted)

def showImages(*kwargs): #shows images
    for k in range(len(kwargs)):
        cv2.imshow(f"Image: {k}", k)
        cv2.waitKey(-1)

def getResidualMetric(residualFrame):
    """Calculate residual metric from average of sum of absolute residual values in residual frame"""
    return np.sum(np.abs(residualFrame))/(residualFrame.shape[0]*residualFrame.shape[1])

def preprocess(anchor, target, blockSize):

    if isinstance(anchor, str) and isinstance(target, str):
        anchorFrame = BGR2YCrCb(cv2.imread(anchor))[:, :, 0] # get luma component
        targetFrame = BGR2YCrCb(cv2.imread(target))[:, :, 0] # get luma component

    elif isinstance(anchor, np.ndarray) and isinstance(target, np.ndarray):
        anchorFrame = BGR2YCrCb(anchor)[:, :, 0] # get luma component
        targetFrame = BGR2YCrCb(target)[:, :, 0] # get luma component

    else:
        raise ValueError

    #resize frame to fit segmentation
    hSegments, wSegments = segmentImage(anchorFrame, blockSize)
    anchorFrame = cv2.resize(anchorFrame, (int(wSegments*blockSize), int(hSegments*blockSize)))
    targetFrame = cv2.resize(targetFrame, (int(wSegments*blockSize), int(hSegments*blockSize)))

    #if debug:
        #print(f"A SIZE: {anchorFrame.shape}")
        #print(f"T SIZE: {targetFrame.shape}")


    return (anchorFrame, targetFrame)

# def main(anchorFrame, targetFrame, outfile="OUTPUT", saveOutput=True, blockSize = 16):
#     """
#     Calculate residual frame and metric along with other artifacts
#     :param anchor: file path of I-Frame or I-Frame
#     :param target: file path of Current Frame or Current Frame
#     :return: residual metric
#     """
#     anchorFrame, targetFrame = preprocess(anchorFrame, targetFrame, blockSize) #processes frame or filepath to frame

#     predictedFrame = blockSearchBody(anchorFrame, targetFrame, blockSize)
#     residualFrame = getResidual(targetFrame, predictedFrame)
#     naiveResidualFrame = getResidual(anchorFrame, targetFrame)
#     reconstructTargetFrame = getReconstructTarget(residualFrame, predictedFrame)
#     #showImages(targetFrame, predictedFrame, residualFrame)

#     residualMetric = getResidualMetric(residualFrame)
#     naiveResidualMetric = getResidualMetric(naiveResidualFrame)

#     rmText = f"Residual Metric: {residualMetric:.2f}"
#     nrmText = f"Naive Residual Metric: {naiveResidualMetric:.2f}"

#     isdir = os.path.isdir(outfile)
#     if not isdir:
#         os.mkdir(outfile)

#     if saveOutput:
#         cv2.imwrite(f"{outfile}/targetFrame.png", targetFrame)
#         cv2.imwrite(f"{outfile}/predictedFrame.png", predictedFrame)
#         cv2.imwrite(f"{outfile}/residualFrame.png", residualFrame)
#         cv2.imwrite(f"{outfile}/reconstructTargetFrame.png", reconstructTargetFrame)
#         cv2.imwrite(f"{outfile}/naiveResidualFrame.png", naiveResidualFrame)
#         resultsFile = open(f"{outfile}/results.txt", "w"); resultsFile.write(f"{rmText}\n{nrmText}\n"); resultsFile.close()

#     print(rmText)
#     print(nrmText)

#     return residualMetric, residualFrame


In [274]:

def read_jpg_images(folder_path):
    # List all files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.jpg')]

    # Sort the image files based on their names
    image_files.sort()

    # Read each JPG image and store them in a list
    images = []
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = cv2.imread(image_path)
        images.append(img)

    return images


def plot_images(image_list):
    for i, img in enumerate(image_list):
        cv2_imshow( img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()


jpg_folder_path ='/content/frames_jpg_8'

images = read_jpg_images(jpg_folder_path)


In [275]:
#-------------------------------PSNR------------------------------------------------#
'''
PSNR>30: Compressed image is with high quality
20<PSNR<30: Compressed image is in medium quality
PSNR<20: Compresses image is in low quality

'''

def psnr(original, compressed):
    # Ensure both images have the same shape
    assert original.shape == compressed.shape, "Images must have the same dimensions"

    # Compute Mean Squared Error (MSE)
    mse = np.mean((original - compressed) ** 2)

    # Compute Peak Signal Power (PSP)
    psp = np.max(original) ** 2

    # Compute PSNR
    psnr_value = 10 * np.log10(psp / mse)

    return psnr_value
# Assuming 'original_image' and 'compressed_image' are NumPy arrays
# psnr_value = psnr(gray_image, inverse_dct_result)
# print(f"PSNR: {psnr_value} dB")


In [276]:
def split_into_blocks(matrix, block_size=8):
    rows, cols = matrix.shape
    block_matrices = []

    for row in range(0, rows, block_size):
        for col in range(0, cols, block_size):
            block = matrix[row:row+block_size, col:col+block_size]
            block_matrices.append(block)

    return block_matrices

In [277]:
def quantize(block, quality):
    # Quality levels for JPEG (Q50, Q70, Q90)
    quality_factors = np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99]
    ])

    if quality < 40:
        scale = 40 / quality
        quality_level = "Low Quality"
    elif 40 <= quality <= 70:
        scale = 2 - quality / 70
        quality_level = "Medium Quality"
    else:
        scale = 100 / quality
        quality_level = "High Quality"

    quantized_block = np.round(block / ((quality_factors+100)* scale))

    return quantized_block, quality_level

def dequantize(quantized_block, quality):
    # Quality levels for JPEG (Q50, Q70, Q90)
    quality_factors = np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99]
    ])

    if quality < 40:
        scale = 40 / quality
    elif 40 <= quality <= 70:
        scale = 2 - quality / 70
    else:
        scale = 100 / quality

    return quantized_block * ((quality_factors+100) * scale)


In [278]:
import heapq
from collections import defaultdict

def calculate_frequencies(arr):
    frequencies = defaultdict(int)
    for row in arr:
        for col in row:
            for value in col:
                frequencies[value] += 1
    return frequencies

def build_huffman_tree(frequencies):
    heap = [[weight, [value, ""]] for value, weight in frequencies.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return heap[0][1:]

def generate_huffman_code(tree):
    huffman_code = {}
    for value, code in tree:
        huffman_code[value] = code
    return huffman_code

def huffman_encode(arr, huffman_code):
    encoded_arr = [[[huffman_code[value] for value in col] for col in row] for row in arr]
    return encoded_arr

def huffman_decode(encoded_arr, huffman_code):
    decoding_dict = {code: value for value, code in huffman_code.items()}
    decoded_arr = [[[decoding_dict[code] for code in col] for col in row] for row in encoded_arr]
    return decoded_arr


In [279]:
def calculate_total_bits(encoded_arr):
    total_bits = 0
    for row in encoded_arr:
        for col in row:
            for code in col:
                total_bits += len(code)
    return total_bits


In [280]:
def write_encoded_data_to_file(encoded_arr, output_file_path):
    with open(output_file_path, 'w') as file:
        for row in encoded_arr:
            for col in row:
                for code in col:
                    file.write(code)


In [281]:
def write_list_to_bin(data_list, output_path):
    # Open the binary file in write mode
    with open(output_path, 'wb') as bin_file:
        # Serialize and write the list to the binary file
        pickle.dump(data_list, bin_file)


In [282]:
def read_list_from_bin(input_path):
    # Open the binary file in read mode
    with open(input_path, 'rb') as bin_file:
        # Deserialize and read the list from the binary file
        data_list = pickle.load(bin_file)

    return data_list


In [283]:
#-------------------------Combining macro blocks-------------------------------------#
def reconstruct_image(blocks, image_shape):
    rows, cols = image_shape
    block_size = blocks[0].shape[0]
    reconstructed_image = np.zeros((rows, cols), dtype=np.float32)

    block_index = 0
    for row in range(0, rows, block_size):
        for col in range(0, cols, block_size):
            reconstructed_image[row:row+block_size, col:col+block_size] = blocks[block_index]
            block_index += 1

    return reconstructed_image


In [284]:
def create_video(images, output_video_path, fps=10):
    # Get the height and width of the images
    height, width, _ = images[0].shape

    # Define the codec and create a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' for AVI format
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Write each frame to the video file
    for frame in images:
        video_writer.write(frame)

    # Release the video writer object
    video_writer.release()

In [285]:
cnt=0
n=0
blockSize=16

# Specify the folder to save the images
output_folder = '/content/Processed_frames_8'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

for img in images:
  # print("1")
  if cnt%5==0:
    # print("2")
    anchorFrame, targetFrame = preprocess(img, img, blockSize) #processes frame
    img_I=img
    # print("3")
    output_file_path=os.path.join(output_folder, f'frame_{cnt:04d}.jpg')
    cv2.imwrite(output_file_path, anchorFrame)
  else:
    # print("4")
    anchorFrame, targetFrame = preprocess(img_I, img, blockSize) #processes frame
    predictedFrame = blockSearchBody(anchorFrame, targetFrame, blockSize)
    img_R = getResidual(anchorFrame, predictedFrame)
    # print("5")
    output_file_path=os.path.join(output_folder,f'frame_{cnt:04d}.jpg')
    cv2.imwrite(output_file_path, img_R)
  cnt+=1




In [286]:
# Specify the folder to save the images
output_folder = '/content/reconstructed_frames_8'
# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
# output_file_path=os.path.join(output_folder,f'frame_{n:04d}.jpg')
# cv2.imwrite(output_file_path, img_R)

In [287]:
jpg_folder_path='/content/Processed_frames_8'
frms_processed = read_jpg_images(jpg_folder_path)

# for frame in frms_processed:
  # cv2_imshow(frame)

In [288]:
n=0
frm_I=[]
for frame in frms_processed:
  gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  original_im_shape=gray_image.shape
  # cv2_imshow(gray_image)
  # print(original_im_shape)

  dct_result = cv2.dct(np.float32(gray_image))
  blocks = split_into_blocks(dct_result, block_size=8)

  quantized_dct_high, quality_level_high = quantize(blocks, 100)
  # print(quantized_dct_high[0:2])

  input_arr=quantized_dct_high
  # Calculate frequencies
  frequencies = calculate_frequencies(input_arr)
  # Build Huffman tree
  huffman_tree = build_huffman_tree(frequencies)
  # Generate Huffman code
  huffman_code = generate_huffman_code(huffman_tree)
  # Huffman encode the input array
  encoded_arr = huffman_encode(input_arr, huffman_code)
  # # Huffman decode the encoded array
  # decoded_arr = huffman_decode(encoded_arr, huffman_code)

  # Display the results
  # print("Huffman Code:", huffman_code)
  # print("Encoded Array:", encoded_arr[0:8])
  # print(type(encoded_arr[0]))
  # # print("Decoded Array:", decoded_arr)
  # print(huffman_code)

  # Calculate total bits for the encoded array
  total_bits = calculate_total_bits(encoded_arr)
  # Display the result
  print("Total Bits:", total_bits)
  # file_size=total_bits/1024
  # print(file_size,"kB")

  output_file_path = 'encoded_data.txt'
  write_encoded_data_to_file(encoded_arr, output_file_path)
  bin_file_path = 'my_list.bin'

  # Write the list to a binary file
  write_list_to_bin(encoded_arr, bin_file_path)

  # # Read the list from the binary file
  # read_list = read_list_from_bin(bin_file_path)

  # print("Original List:", my_list)
  # print("Read List:", read_list)

  #------------------------Reconstruction-------------------#

   # Read the list from the binary file
  read_list = read_list_from_bin(bin_file_path)

  decoded_arr = huffman_decode(read_list  , huffman_code)
  # print(decoded_arr)

  dequantized_blocks=[]
  for blocks in decoded_arr:
      # Filter out None values
    filtered_array = np.where(blocks != None, blocks, np.nan)

    # Convert the filtered array to a numeric type (float)
    filtered_array = filtered_array.astype(np.float32)
    dequantized_block=dequantize(filtered_array, 100)
    dequantized_blocks.append(dequantized_block)
  # print(dequantized_blocks[0:2])

  merged_img=reconstruct_image(dequantized_blocks, original_im_shape)

  inverse_dct_result = cv2.idct(np.float32(merged_img))
  # print(inverse_dct_result[0:3])

  # cv2_imshow(inverse_dct_result)

  if n%5==0:
    frm_I=inverse_dct_result
    output_file_path=os.path.join(output_folder,f'frame_{n:04d}.jpg')
    cv2.imwrite(output_file_path, frm_I)
  else:
    residualFrame=inverse_dct_result
    reconstructTargetFrame = getReconstructTarget(residualFrame, frm_I)
    output_file_path=os.path.join(output_folder,f'frame_{n:04d}.jpg')
    cv2.imwrite(output_file_path, reconstructTargetFrame)
  n+=1
  psnr_value = psnr(gray_image, inverse_dct_result)
  print(f"PSNR: {psnr_value} dB")

Total Bits: 2227882
PSNR: 29.504930635342703 dB
Total Bits: 2208879
PSNR: 29.261909193678264 dB
Total Bits: 2210537
PSNR: 29.078693915887786 dB
Total Bits: 2209906
PSNR: 29.186926179286473 dB
Total Bits: 2209947
PSNR: 29.21139093895025 dB
Total Bits: 2228721
PSNR: 29.477858779082247 dB
Total Bits: 2206402
PSNR: 29.59631319852771 dB
Total Bits: 2208904
PSNR: 29.367578169968354 dB
Total Bits: 2210121
PSNR: 29.16139644352491 dB
Total Bits: 2209352
PSNR: 29.174560307278945 dB
Total Bits: 2227176
PSNR: 29.79576666821382 dB
Total Bits: 2206015
PSNR: 29.735663614322462 dB
Total Bits: 2208382
PSNR: 29.505612679430243 dB
Total Bits: 2209302
PSNR: 29.326924879982176 dB
Total Bits: 2209283
PSNR: 29.35694988188327 dB
Total Bits: 2228467
PSNR: 29.606740594913738 dB
Total Bits: 2208607
PSNR: 29.35732429178654 dB
Total Bits: 2210960
PSNR: 29.18982420573205 dB
Total Bits: 2210757
PSNR: 29.291214709351458 dB
Total Bits: 2211128
PSNR: 29.228793276249174 dB
Total Bits: 2230127
PSNR: 29.281724508080124 dB

In [289]:
jpg_folder_path = '/content/reconstructed_frames_8'
output_video_path = '/content/output.mp4'

jpg_images = read_jpg_images(jpg_folder_path)

# Create a video from the images
create_video(jpg_images, output_video_path)